In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
#from tensorflow_addons import losses
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
df = pd.read_csv("customer_churn.csv")
df.drop('customerID', axis='columns', inplace=True)
df[pd.to_numeric(df.TotalCharges, errors='coerce').isnull()]
df1 = df[df.TotalCharges!=' ']
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)


In [5]:
for column in df1:
    print(f'{column} : {df1[column].unique()}')

gender : ['Female' 'Male']
SeniorCitizen : [0 1]
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService : ['No' 'Yes']
MultipleLines : ['No phone service' 'No' 'Yes']
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : ['No' 'Yes' 'No internet service']
OnlineBackup : ['Yes' 'No' 'No internet service']
DeviceProtection : ['No' 'Yes' 'No internet service']
TechSupport : ['No' 'Yes' 'No internet service']
StreamingTV : ['No' 'Yes' 'No internet service']
StreamingMovies : ['No' 'Yes' 'No internet service']
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : ['Yes' 'No']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78

In [7]:
df1.replace('No internet service', 'No', inplace=True) #inplace means the chosen df is affected by the function
df1.replace('No phone service', 'No', inplace=True)
yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                  'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

In [8]:
for column in df1:
        if df1[column].dtypes=='object':
            print(f'{column} : {df1[column].unique()}')

gender : ['Female' 'Male']
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
PhoneService : ['No' 'Yes']
MultipleLines : ['No' 'Yes']
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : ['No' 'Yes']
OnlineBackup : ['Yes' 'No']
DeviceProtection : ['No' 'Yes']
TechSupport : ['No' 'Yes']
StreamingTV : ['No' 'Yes']
StreamingMovies : ['No' 'Yes']
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : ['Yes' 'No']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn : ['No' 'Yes']


In [10]:
for col in yes_no_columns:
    df1[col].replace({'Yes': 1, 'No': 0}, inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [11]:
for column in df1:
        if df1[column].dtypes=='object':
            print(f'{column} : {df1[column].unique()}')

gender : ['Female' 'Male']
InternetService : ['DSL' 'Fiber optic' 'No']
Contract : ['Month-to-month' 'One year' 'Two year']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [12]:
df1['gender'].replace({'Female': 1, 'Male': 0}, inplace=True)

In [13]:
df2 = pd.get_dummies(data=df1, columns=['InternetService', 'Contract', 'PaymentMethod'])
cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])
X = df2.drop('Churn', axis='columns')
y = df2['Churn']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [22]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    
    model = keras.Sequential([
    keras.layers.Dense(26, input_dim=26, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
        ])
    
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [24]:
y_test.value_counts()

0    999
1    408
Name: Churn, dtype: int64

In [23]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 1s 828us/step - loss: 0.4939 - accuracy: 0.7501
Epoch 2/100
176/176 [==============================] - 0s 896us/step - loss: 0.4245 - accuracy: 0.7966
Epoch 3/100
176/176 [==============================] - 0s 940us/step - loss: 0.4244 - accuracy: 0.7978
Epoch 4/100
176/176 [==============================] - 0s 866us/step - loss: 0.4151 - accuracy: 0.8032
Epoch 5/100
176/176 [==============================] - 0s 909us/step - loss: 0.4178 - accuracy: 0.8036
Epoch 6/100
176/176 [==============================] - 0s 884us/step - loss: 0.4092 - accuracy: 0.8125
Epoch 7/100
176/176 [==============================] - 0s 909us/step - loss: 0.3993 - accuracy: 0.8121
Epoch 8/100
176/176 [==============================] - 0s 845us/step - loss: 0.4102 - accuracy: 0.8119
Epoch 9/100
176/176 [==============================] - 0s 834us/step - loss: 0.4173 - accuracy: 0.7999
Epoch 10/100
176/176 [==============================] - 0s 829us/step - l

176/176 [==============================] - 0s 925us/step - loss: 0.3443 - accuracy: 0.8348
Epoch 81/100
176/176 [==============================] - 0s 924us/step - loss: 0.3470 - accuracy: 0.8336
Epoch 82/100
176/176 [==============================] - 0s 884us/step - loss: 0.3482 - accuracy: 0.8369
Epoch 83/100
176/176 [==============================] - 0s 861us/step - loss: 0.3552 - accuracy: 0.8255
Epoch 84/100
176/176 [==============================] - 0s 921us/step - loss: 0.3402 - accuracy: 0.8382
Epoch 85/100
176/176 [==============================] - 0s 890us/step - loss: 0.3393 - accuracy: 0.8415
Epoch 86/100
176/176 [==============================] - 0s 897us/step - loss: 0.3415 - accuracy: 0.8373
Epoch 87/100
176/176 [==============================] - 0s 933us/step - loss: 0.3501 - accuracy: 0.8373
Epoch 88/100
176/176 [==============================] - 0s 963us/step - loss: 0.3385 - accuracy: 0.8458
Epoch 89/100
176/176 [==============================] - 0s 883us/step - loss:

## Sampling

In [26]:
# Class Count
count_class_0, count_class_1 = df1.Churn.value_counts()

# Divide by class
df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [27]:
df_class_0.shape

(5163, 27)

In [28]:
df_class_1.shape

(1869, 27)

## Undersampling

In [32]:
# Getting creating proper sample size through count
df_class_0_under = df_class_0.sample(count_class_1)

df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under_sample:')
print(df_test_under.Churn.value_counts())

Random under_sample:
1    1869
0    1869
Name: Churn, dtype: int64


In [33]:
X = df_test_under.drop('Churn', axis='columns')
y = df_test_under['Churn']

# Stratify makes sure there are balanced samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5, stratify=y)

In [34]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [35]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)
# Notice improved f-1 score, because the data is now balanced

Epoch 1/100
94/94 [==============================] - 0s 723us/step - loss: 0.7027 - accuracy: 0.5670
Epoch 2/100
94/94 [==============================] - 0s 590us/step - loss: 0.5429 - accuracy: 0.7476
Epoch 3/100
94/94 [==============================] - 0s 525us/step - loss: 0.4924 - accuracy: 0.7708
Epoch 4/100
94/94 [==============================] - 0s 525us/step - loss: 0.4869 - accuracy: 0.7642
Epoch 5/100
94/94 [==============================] - 0s 530us/step - loss: 0.4603 - accuracy: 0.7728
Epoch 6/100
94/94 [==============================] - 0s 536us/step - loss: 0.4805 - accuracy: 0.7601
Epoch 7/100
94/94 [==============================] - 0s 525us/step - loss: 0.4704 - accuracy: 0.7752
Epoch 8/100
94/94 [==============================] - 0s 558us/step - loss: 0.4741 - accuracy: 0.7567
Epoch 9/100
94/94 [==============================] - 0s 536us/step - loss: 0.4774 - accuracy: 0.7647
Epoch 10/100
94/94 [==============================] - 0s 556us/step - loss: 0.4481 - accura

Epoch 81/100
94/94 [==============================] - 0s 536us/step - loss: 0.3762 - accuracy: 0.8343
Epoch 82/100
94/94 [==============================] - 0s 525us/step - loss: 0.3689 - accuracy: 0.8433
Epoch 83/100
94/94 [==============================] - 0s 528us/step - loss: 0.3566 - accuracy: 0.8383
Epoch 84/100
94/94 [==============================] - 0s 553us/step - loss: 0.3589 - accuracy: 0.8254
Epoch 85/100
94/94 [==============================] - 0s 525us/step - loss: 0.3614 - accuracy: 0.8395
Epoch 86/100
94/94 [==============================] - 0s 536us/step - loss: 0.3621 - accuracy: 0.8368
Epoch 87/100
94/94 [==============================] - 0s 525us/step - loss: 0.3621 - accuracy: 0.8387
Epoch 88/100
94/94 [==============================] - 0s 612us/step - loss: 0.3661 - accuracy: 0.8329
Epoch 89/100
94/94 [==============================] - 0s 525us/step - loss: 0.3667 - accuracy: 0.8351
Epoch 90/100
94/94 [==============================] - 0s 525us/step - loss: 0.3700